In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def r2_prediction(regressor, X_test, y_test):
    y_pred = regressor.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

def Linear(X_train, y_train, X_test, y_test):
    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_linear(X_train, y_train, X_test, y_test):
    regressor = SVR(kernel='linear')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def svm_NL(X_train, y_train, X_test, y_test):
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def Decision(X_train, y_train, X_test, y_test):
    regressor = DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def random(X_train, y_train, X_test, y_test):
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    lin = LinearRegression()
    SVRl = SVR(kernel='linear')
    dec = DecisionTreeRegressor(random_state=0)
    rf = RandomForestRegressor(n_estimators=10, random_state=0)
    rfemodellist = [lin, SVRl, dec, rf]
    for i in rfemodellist:
        log_rfe = RFE(estimator=i, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)
    return rfelist

def rfe_regression(acclog, accsvml, accdes, accrf):
    rfedataframe = pd.DataFrame(index=['Linear', 'SVMl', 'Decision', 'Random'],
                                 columns=['Linear', 'SVMl', 'Decision', 'Random'])

    for number, idex in enumerate(rfedataframe.index):
        rfedataframe['Linear'][idex] = acclog[number]
        rfedataframe['SVMl'][idex] = accsvml[number]
        rfedataframe['Decision'][idex] = accdes[number]
        rfedataframe['Random'][idex] = accrf[number]
    return rfedataframe

# Assuming 'df2' is your DataFrame
dataset1 = pd.read_csv("prep.csv", index_col=None)
df2 = dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

rfelist = rfeFeature(indep_X, dep_Y, 3)

acclin = []
accsvml = []
accsvmnl = []
accdes = []
accrf = []

for i in rfelist:
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    r2_lin = Linear(X_train, y_train, X_test, y_test)
    acclin.append(r2_lin)

    r2_sl = svm_linear(X_train, y_train, X_test, y_test)
    accsvml.append(r2_sl)

    r2_NL = svm_NL(X_train, y_train, X_test, y_test)
    accsvmnl.append(r2_NL)

    r2_d = Decision(X_train, y_train, X_test, y_test)
    accdes.append(r2_d)

    r2_r = random(X_train, y_train, X_test, y_test)
    accrf.append(r2_r)

result = rfe_regression(acclin, accsvml, accdes, accrf)
result

,Linear,SVMl,Decision,Random
Linear,0.441961,0.262153,0.441961,0.441816
SVMl,0.441961,0.262153,0.441961,0.441816
Decision,0.664893,0.609652,0.965961,0.916304
Random,0.676174,0.670691,0.933504,0.887256
